In [2]:
import numpy
import pandas as pd
from pandas import Series, DataFrame
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# commands need to link to kaggle
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"heeseokjung","key":"6269215eb20f89c9e71345554a3ffae0"}'}

In [22]:
# commands need to link to kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [23]:
# dataset downloat link
! kaggle competitions download -c titanic
# if unzip needed
# !unzip

train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)


In [27]:
train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./test.csv")


train_data.info()
print('*' * 20)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
********************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    

In [28]:
train_data = train_data.drop(["PassengerId", "Name", "Ticket", "SibSp", "Parch", "Fare", "Cabin", "Embarked"], axis=1)
test_data = test_data.drop(["Name", "Ticket", "SibSp", "Parch", "Fare", "Cabin", "Embarked"], axis=1)

In [29]:
pclass_train_dummies = pd.get_dummies(train_data["Pclass"])
pclass_test_dummies = pd.get_dummies(test_data["Pclass"])

train_data.drop(["Pclass"], axis=1, inplace=True)
test_data.drop(["Pclass"], axis=1, inplace=True)

train_data = train_data.join(pclass_train_dummies)
test_data = test_data.join(pclass_test_dummies)

In [30]:
sex_train_dummies = pd.get_dummies(train_data["Sex"])
sex_test_dummies = pd.get_dummies(test_data["Sex"])

sex_train_dummies.columns = ["Female", "Male"]
sex_test_dummies.columns = ["Female", "Male"]

train_data.drop(["Sex"], axis=1, inplace=True)
test_data.drop(["Sex"], axis=1, inplace=True)

train_data = train_data.join(sex_train_dummies)
test_data = test_data.join(sex_test_dummies)

In [31]:
train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
test_data["Age"].fillna(train_data["Age"].mean(), inplace=True)

In [32]:
x_train = train_data.drop("Survived", axis=1)
y_train = train_data["Survived"]
x_test = test_data.drop("PassengerId", axis=1).copy()

In [33]:
knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

knn.score(x_train, y_train)

0.8372615039281706

In [34]:
submission = pd.DataFrame({"PassengerId": test_data["PassengerId"],
                           "Survived": y_pred})
submission.to_csv("submission.csv", index=False)

In [35]:
# submit result
#!kaggle competitions submit -c titanic -f submission.csv -m "titanic"

100% 2.77k/2.77k [00:09<00:00, 299B/s]
Successfully submitted to Titanic - Machine Learning from Disaster